In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras
from typing import Dict, Tuple
import re
import keras.layers as l
from keras import models, callbacks, utils, losses
import string

2024-05-26 13:19:00.507811: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-26 13:19:01.258254: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
valid_letters = set("ёйцукенгшщзхъфывапролджэячсмитьбюЁЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮ" + string.printable)
dirname = "books"

text = []
for subdir in os.listdir(dirname):
    for text_file in os.listdir(f"{dirname}/{subdir}"):
        if not re.match(r"[a-zA-Z_0-9]+.txt", text_file):
            continue
        with open(f"{dirname}/{subdir}/{text_file}") as f:
            text.extend([ch for ch in f.read() if ch in valid_letters])
text = "".join(text)[:2_000_000]

In [4]:
def get_features_target(seq: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 32

words = list(filter(None, [re.sub('[^а-яА-ЯёЁ0-9 ,-]', '', s).strip() for s in text.split('.')]))
alphabet = np.array(sorted(set(' '.join(words).split(' '))))

word_index = {char: i for i, char in enumerate(alphabet)}
index_word = {i: char for i, char in enumerate(alphabet)}

sequences = Dataset.from_tensor_slices(np.array([word_index[word] for word in ' '.join(words).split()])).batch(BATCH_SIZE, drop_remainder=True)
dataset = sequences.map(get_features_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)


2024-05-26 13:22:18.278711: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79262 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:8c:00.0, compute capability: 8.0


In [10]:
model = keras.Sequential([
    l.Embedding(len(alphabet), BATCH_SIZE, batch_input_shape=[BATCH_SIZE, None]),
    l.LSTM(512, return_sequences=True, stateful=True),
    l.Dense(len(alphabet) / 2, activation='relu'),
    l.Dense(len(alphabet))
])

model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(data, epochs=35, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

2024-05-26 13:53:11.040279: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-26 13:53:11.041668: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-26 13:53:11.042906: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/35


2024-05-26 13:53:11.241902: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-26 13:53:11.244460: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-26 13:53:11.245962: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

299/299 [==============================] - 54s 174ms/step - loss: 8.8210 - accuracy: 0.0309
Epoch 2/35
299/299 [==============================] - 51s 172ms/step - loss: 7.7430 - accuracy: 0.0516
Epoch 3/35
299/299 [==============================] - 52s 172ms/step - loss: 7.0704 - accuracy: 0.0667
Epoch 4/35
299/299 [==============================] - 51s 172ms/step - loss: 6.4844 - accuracy: 0.0793
Epoch 5/35
299/299 [==============================] - 51s 172ms/step - loss: 6.1703 - accuracy: 0.0902
Epoch 6/35
299/299 [==============================] - 51s 172ms/step - loss: 5.8159 - accuracy: 0.1071
Epoch 7/35
299/299 [==============================] - 51s 172ms/step - loss: 5.4185 - accuracy: 0.1270
Epoch 8/35
299/299 [==============================] - 51s 172ms/step - loss: 5.0489 - accuracy: 0.1585
Epoch 9/35
299/299 [==============================] - 51s 172ms/step - loss: 4.5425 - accuracy: 0.2109
Epoch 10/35
299/299 [==============================] - 51s 172ms/step - loss: 3.9609

In [11]:
def predict_next(sample: str, model: keras.Sequential, tokenizer: Dict[str, int], vocabulary: Dict[int, str], n_next: int, temperature: float, batch_size: int, word: bool = False) -> str:
    if word:
        sample_vector = [tokenizer[word] for word in sample.split()]
    else:
        sample_vector = [tokenizer[char] for char in sample]
    predicted = sample_vector
    sample_tensor = tf.expand_dims(sample_vector, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    for i in range(n_next):
        pred = model(sample_tensor)
        pred = pred[0].numpy() / temperature
        pred = tf.random.categorical(pred, num_samples=1)[-1, 0].numpy()
        predicted.append(pred)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([pred], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    pred_seq = [vocabulary[i] for i in predicted]
    generated = ' '.join(pred_seq) if word else ''.join(pred_seq)
    return generated

In [19]:
print(predict_next(
    sample='Что',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=64,
    temperature=0.95,
    batch_size=BATCH_SIZE,
    word=True
))

Что же девушка в руках Но тот самый спичечный коробок, ради колени и под этими листьями обнаженная женщина, снаружи остаются лишь руки и ноги налетает ветер, и открывается лицо и лицо это превращается неожиданно на лицо он уже не от ног Меня Смысл Однако тот самый добычу меня с пианино и это делает их Или же хоть ее для нее в этом случае, который не


In [18]:
print(predict_next(
    sample='Где',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=64,
    temperature=0.9,
    batch_size=BATCH_SIZE,
    word=True
))

Где я не думаю, что он не в силах сомкнуть глаз, поэтому хотя я проснулся совсем недавно, так и не смог выспаться как следует Но это время еще ничего когда я проснулся, то обнаружил, что белье и брюки в два с половиной раза в нем установлено множество телефонов-автоматов По рассказам очевидцев, этот человек примерно полдня сидел в одной и той же позе, но в течение


In [20]:
print(predict_next(
    sample='Когда',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=64,
    temperature=0ю8,
    batch_size=BATCH_SIZE,
    word=True
))

Когда если ты не был на свою силы и, чтобы скрыть свое вины, безвозвратно один человек, и с таким смешением Предвкушая жареную креветку, человек выплюнет еду и меня он с такой глупости, чтобы, в нее отходы только потому, что в него нет Если бы было ни денег, то, может быть, мне бы я не в силах больше ждать Действительно, ждать невыносимо И тем не менее
